In [0]:
import dlt
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests
import json

spark = SparkSession.builder \
    .appName("Example") \
    .getOrCreate()

In [0]:
latest_readings_API = requests.get("http://environment.data.gov.uk/flood-monitoring/data/readings?latest")
latest_readings_json = json.loads(latest_readings_API.text)
latest_readings_json = latest_readings_json['items']

@dlt.table(
  comment="Raw data of latest readings."
)
def latest_readings_raw():          
  return (
    spark.read.json(spark.sparkContext.parallelize([latest_readings_json]))
  )

In [0]:
@dlt.table(
  comment="Latest readings data cleaned and refreshed"
)

def latest_readings_prepared():
  return (
    dlt.read("latest_readings_raw")
      .withColumn("measure_before_slash", split(col("measure"),"/")[0])
      .withColumn("measure_after_slash", split(col("measure"),"/")[1])
      .select("@id", "measure", "measure_before_slash","measure_after_slash","dateTime","value")
  )